## Import bibliotek.

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('whitegrid')

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import tree

## Ćwiczenie 1.
Wczytajamy zbiory dotyczące danych sprzedażowych "Produktu A".

In [2]:
tr = pd.read_parquet('dane/tr.parquet')
va = pd.read_parquet('dane/va.parquet')

#### 1.1. Rzuć okiem na zbiór. Odrzuć niepotrzebne kolumny.

**Zanim zaczniesz, przeczytaj proszę wnioski z poprzednich zajęć, które podsumowałem poniżej.**

---
Dla przypomnienia, wnioski z poprzednich zajęć i analizy zbioru uczącego:
* Najmocniej skorelowana ze zmienną celu: "srednia_sprzedaz_0_13d" - 0.783053
* Ze zmienną "srednia_sprzedaz_0_13d" wysoce skorelowane są zmienne: srednia_sprzedaz_0_2d, srednia_sprzedaz_0_6d
* Zmienne do sprawdzenia:
    * "srednia_cena_ropy_0_6d" - mocniej skorelowana ze zmienną celu niż srednia_cena_ropy_0_29d
    * "rok_dzialalnosci" - z każdym rokiem sprzedaż nieco wzrasta.
    * Pozostałe zmienne oparte na dacie: święta, weekendy, etc.
---

Skorzystaj z metody `tr.drop(columns=[], inplace=True)`

PS. Pamiętaj, by identyczą operację przeprowadzić dla obu zbiorów.

In [3]:
to_drop = ['srednia_sprzedaz_0_2d', 'data_sprzedazy', 'data_odciecia', 'cena_ropy', 'dzien_tygodnia']

In [4]:
tr.drop(columns=to_drop, inplace=True)
va.drop(columns=to_drop, inplace=True)

#### 1.2. Zbiór będzie zawierać wiele zmiennych. Rozdzielmy więc X i y na dwa zbiory.
Utwórz zmienne `X_tr`, `X_va`, `y_tr` i `y_va`.

In [5]:
X_tr = tr.drop(columns='sprzedaz').copy()
X_va = va.drop(columns='sprzedaz').copy()

y_tr = tr.sprzedaz.copy()
y_va = va.sprzedaz.copy()

In [6]:
X_tr.head()

,id_drogerii,miasto,wojewodztwo,typ_drogerii,srednia_sprzedaz_0_6d,srednia_sprzedaz_0_13d,srednia_cena_ropy_0_6d,srednia_cena_ropy_0_29d,weekend,swieto,miesiac,rok_dzialalnosci,sprzedaz_minus_1t
36,1,Warszawa,Mazowieckie,D,2.428571,2.214286,96.050000,94.531667,False,False,Luty,1,97.98
37,1,Warszawa,Mazowieckie,D,2.428571,2.142857,96.378571,94.682000,False,False,Luty,1,97.65
38,1,Warszawa,Mazowieckie,D,2.142857,2.357143,96.708571,94.826000,False,False,Luty,1,97.46
39,1,Warszawa,Mazowieckie,D,1.571429,2.000000,97.038571,94.975667,True,False,Luty,1,97.46
40,1,Warszawa,Mazowieckie,D,1.285714,1.785714,97.368571,95.120333,True,False,Luty,1,97.46


#### 1.3.Wykonaj konwersję zmiennych kategorycznych do typu binarnego.
* Skorzystaj z funkcji `pd.get_dummies(X_tr, columns=[tu_wpisz_zmienne_do_przekonwertowania])`.
* Wykonaj konwersję dla zmiennych: `['typ_drogerii', 'weekend', 'swieto', 'miasto', 'wojewodztwo', 'id_drogerii', 'miesiac']`.
* Wykonaj konwersję zarówno dla zbioru X_tr, jak i X_va.

In [7]:
X_tr = pd.get_dummies(X_tr, columns=['typ_drogerii', 'weekend', 'swieto', 'miasto', 'wojewodztwo', 'id_drogerii', 'miesiac'], drop_first=True)
X_va = pd.get_dummies(X_va, columns=['typ_drogerii', 'weekend', 'swieto', 'miasto', 'wojewodztwo', 'id_drogerii', 'miesiac'], drop_first=True)

## Ćwiczenie 2.

#### 2.1. Zbuduj prostu "niedouczony" model drzewa decyzyjnego. Będzie on naszym punktem odniesienia dla kolejnych modeli.
* Ustaw parametry `max_dept=3` i `min_samples_leaf=0.1`.

In [11]:
model_dd = DecisionTreeRegressor(max_depth=3, min_samples_leaf=0.1)
model_dd.fit(X_tr, y_tr)

DecisionTreeRegressor(max_depth=3, min_samples_leaf=0.1)

#### 2.2. Sprawdź jakość modelu na zbiorze `X_va`.

In [17]:
pred_tr_dd = model_dd.predict(X_tr)
pred_va_dd = model_dd.predict(X_va)

score_tr_dd = mean_squared_error(y_tr, pred_tr_dd, squared=False)
score_va_dd = mean_squared_error(y_va, pred_va_dd, squared=False)

print('Drzewo decyzyjne TR: ', np.round(score_tr_dd, 2))
print('Drzewo decyzyjne TE: ', np.round(score_va_dd, 2))

Drzewo decyzyjne TR:  2.63
Drzewo decyzyjne TE:  2.73


Najlepsze wyniki, jakie udało się osiągnąć z użyciem regresji liniowej:
* MSE TR: 2.294
* MSE VA: 2.298

Jest więc miejsce na poprawę.

## Ćwiczenie 3.
#### 3.1. Wykonaj próbkowanie z użyciem metody `df.sample`.
* Zbiór jest dosyć długi (prawie 50 tys. obserwacji) i dosić szeroki (kilkadziesiąt zmiennych). Na potrzeby poszukiwania optymalnych parametrów przeprowadzimy próbkowanie.

In [18]:
X_tr_sampled = X_tr.sample(5000, random_state=2001)
y_tr_sampled = y_tr.loc[X_tr_sampled.index]

#### 3.2. Zdefiniuj siatką parametrów i ustal odpowiednią CV.

In [26]:
parametry = {
    'criterion': ["mse", "friedman_mse"],
    'max_depth': [2, 3, 4, 5],
    'min_samples_split': [0.01, 0.02, 0.05],
    'min_samples_leaf': [10, 15, 20, 30],
    'random_state': [2001]
}
model_cv = DecisionTreeRegressor()

#### 3.3. Naucz nowy model i przeprowadź weryfikację jego jakości.

In [34]:
gs = GridSearchCV(model_cv, parametry, n_jobs=2, scoring='neg_mean_squared_error', cv=3)
gs.fit(X_tr, y_tr)

model_cv = DecisionTreeRegressor(**gs.best_params_)
model_cv.fit(X_tr, y_tr)

pred_tr = model_cv.predict(X_tr)
pred_va = model_cv.predict(X_va)

score_tr = mean_squared_error(y_tr, pred_tr, squared=False)
score_va = mean_squared_error(y_va, pred_va, squared=False)

print('TR: ', np.round(score_tr, 2))
print('TE: ', np.round(score_va, 2))

/opt/jupyterhub/lib/python3.8/site-packages/sklearn/tree/_classes.py:397: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warnings.warn(
/opt/jupyterhub/lib/python3.8/site-packages/sklearn/tree/_classes.py:397: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warnings.warn(


TR:  2.31
TE:  2.33


#### 3.4. Sprawdź jakie zmienne okazaly się istotne.

In [35]:
pd.Series(model_cv.feature_importances_, index=X_tr.columns).sort_values(ascending=False)[0:10]

srednia_sprzedaz_0_13d    0.871868
swieto_True               0.070732
srednia_sprzedaz_0_6d     0.031588
id_drogerii_44            0.015260
weekend_True              0.010529
miasto_Lublin             0.000023
id_drogerii_28            0.000000
id_drogerii_29            0.000000
id_drogerii_30            0.000000
id_drogerii_32            0.000000
dtype: float64

## Ćwiczenie 4.
#### 4.1. Wczytaj zbiór "wages_categorical.csv", który znajduje się w katalogu "dane". Zawiera on mnóstwo zmiennych kategorycznych - idealnie dla drzewa decyzyjnego! :-)

In [ ]:
dane 


#### 4.2. Przeprowadź konwersję zmiennych kategorycznych do kodowania *dummy coding*.

In [ ]:
X_tr = pd.get_dummies(X_tr, columns=['typ_drogerii', 'weekend', 'swieto', 'miasto', 'wojewodztwo', 'id_drogerii', 'miesiac'], drop_first=True)
X_va = pd.get_dummies(X_va, columns=['typ_drogerii', 'weekend', 'swieto', 'miasto', 'wojewodztwo', 'id_drogerii', 'miesiac'], drop_first=True)


#### 4.3. Podziel zbiór na 2 części w proporcjach 2:1. Rozdziel X od y.

In [ ]:
# tu wpisz kod


#### 4.4. Zamodeluj zmienną "wage". Zacznij od bazowego modelu z parametrami `max_depth=3, min_samples_leaf=0.1`.

In [ ]:
# tu wpisz kod


#### 4.5. Poszukaj sub-optymalnych parametrów modelu używając `GridSearchCV`.

In [ ]:
# tu wpisz kod


#### 4.6. Naucz nowy model i przeprowadź weryfikację jego jakości.

In [ ]:
# tu wpisz kod


#### 4.7. Sprawdź jakie zmienne okazaly się istotne.

In [ ]:
# tu wpisz kod
